In [23]:
import pandas as pd
import boto3
import json
import pandas as pd
import io

# the below extension properly formats a cell after it is run
%load_ext nb_black

# Set the maximum number of rows to 200
pd.set_option("display.max_rows", 200)
import pandas as pd

# Set the maximum number of columns to 200
pd.set_option("display.max_columns", 200)

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [24]:


# Creating an S3 client object
s3 = boto3.client('s3')

# Specifying the name of the bucket
bucket_name = 'capstonehaystacks'

# Downloading the all_zips_grocery_store.json file from S3 and parsing it into data1
response1 = s3.get_object(Bucket=bucket_name, Key='all_zips_grocery_store.json')
json_content1 = response1['Body'].read().decode('utf-8')
grocery_json = json.loads(json_content1)

# Downloading the all_zips_restaurant.json file from S3 and parsing it into data2
response2 = s3.get_object(Bucket=bucket_name, Key='all_zips_restaurant.json')
json_content2 = response2['Body'].read().decode('utf-8')
restaurant_json = json.loads(json_content2)


<IPython.core.display.Javascript object>

In [26]:
# # THIS CODE LOADS POI'S WITH NO RATINGS AND ALSO INCLUDES ADDRESS AND ZIP CODE COLUMNS


# def json_to_df(json):
#     """
#     Convert a list of JSON objects containing location information into a Pandas DataFrame.

#     :param json: A list of JSON objects containing location information.
#     :return: A Pandas DataFrame containing the location information.
#     """
#     data = []

#     for j in json:
#         zipcode_ = j['census_zcta_geoid']
#         for result in j['responce']['results']:
#             try:
#                 # extracting the state from the compound code and check if it's 'GA'
#                 state_ = result['plus_code']['compound_code'].split(',')[-2].strip()
#                 poi_types_ = result['types']

#                 if state_ == 'GA':
#                     # extracting relevant information from the JSON object
#                     place_id_ = result['place_id']
#                     latitude_ = result['geometry']['location']['lat']
#                     longitude_ = result['geometry']['location']['lng']
#                     rating_ = result.get('rating', None)
#                     num_ratings_ = result.get('user_ratings_total', None)
#                     name_ = result['name']
#                     address_ = result['vicinity']

#                     # appending the extracted information to the data list
#                     data.append([latitude_, longitude_, state_, place_id_, name_, rating_, num_ratings_, poi_types_, zipcode_, address_])
#             except:
#                 pass

#     # creating df using the data list
#     df = pd.DataFrame(data, columns=['latitude', 'longitude', 'state', 'place_id', 'name', 'rating', 'num_ratings', 'poi_types', 'zipcode', 'address'])
#     return df


<IPython.core.display.Javascript object>

In [30]:
# THIS CODE LOADS ONLY OBSERVATIONS WITH RATINGS

def json_to_df(json):
    """
    Convert a list of JSON objects containing location information into a Pandas DataFrame.

    :param json: A list of JSON objects containing location information.
    :return: A Pandas DataFrame containing the location information.
    """
    data = []

    for j in json:
        zipcode_ = j['census_zcta_geoid']
        for result in j['responce']['results']:
            try:
                # extracting the state from the compound code and check if it's 'GA'
                state_ = result['plus_code']['compound_code'].split(',')[-2].strip()
                poi_types_ = result['types']

                if state_ == 'GA':
                    # extracting relevant information from the JSON object
                    place_id_ = result['place_id']
                    latitude_ = result['geometry']['location']['lat']
                    longitude_ = result['geometry']['location']['lng']
                    rating_ = result.get('rating', None)
                    num_ratings_ = result.get('user_ratings_total', None)
                    name_ = result['name']
                    address_ = result['vicinity']

                    # check if the POI has user reviews
                    if num_ratings_ is not None and num_ratings_ > 0:
                        # appending the extracted information to the data list
                        data.append([latitude_, longitude_, state_, place_id_, name_, rating_, num_ratings_, poi_types_, zipcode_, address_])
            except:
                pass

    # creating df using the data list
    df = pd.DataFrame(data, columns=['latitude', 'longitude', 'state', 'place_id', 'name', 'rating', 'num_ratings', 'poi_types', 'zipcode', 'address'])
    return df


<IPython.core.display.Javascript object>

In [31]:
grocery_df = json_to_df(grocery_json)

<IPython.core.display.Javascript object>

In [32]:
grocery_df.shape

(3190, 10)

<IPython.core.display.Javascript object>

In [33]:
grocery_df.isna().sum()

latitude       0
longitude      0
state          0
place_id       0
name           0
rating         0
num_ratings    0
poi_types      0
zipcode        0
address        0
dtype: int64

<IPython.core.display.Javascript object>

In [34]:
restaurant_df = json_to_df(restaurant_json)

<IPython.core.display.Javascript object>

In [35]:
restaurant_df.shape

(5283, 10)

<IPython.core.display.Javascript object>

In [36]:
poi_df = pd.concat([grocery_df, restaurant_df], axis=0)

<IPython.core.display.Javascript object>

In [37]:
poi_df["primary_type"] = poi_df["poi_types"].apply(lambda x: x[0])

<IPython.core.display.Javascript object>

In [38]:
poi_df.isna().sum()

latitude        0
longitude       0
state           0
place_id        0
name            0
rating          0
num_ratings     0
poi_types       0
zipcode         0
address         0
primary_type    0
dtype: int64

<IPython.core.display.Javascript object>

In [40]:
poi_df

,latitude,longitude,state,place_id,name,rating,num_ratings,poi_types,zipcode,address,primary_type
0,33.791987,-84.385677,GA,ChIJoz4--E8E9YgRxpSO8xC1jj0,Residence Inn Atlanta Midtown/Peachtree at 17th,4.1,723,"[lodging, point_of_interest, establishment]",30309,"1365 Peachtree Street Northeast, Atlanta",lodging
1,33.790216,-84.388776,GA,ChIJh-lI8FoE9YgRyKGKYvLpvA8,Artmore Hotel,3.9,1106,"[lodging, point_of_interest, establishment]",30309,"1302 West Peachtree Street Northwest, Atlanta",lodging
2,33.799427,-84.392589,GA,ChIJFybLXVQE9YgRTePNqN08VnU,Atlanta Peachtree Station,3.6,163,"[train_station, transit_station, point_of_inte...",30309,"1688 Peachtree Road Northwest, Atlanta",train_station
3,33.797403,-84.387181,GA,ChIJw7eoDVIE9YgRx5tdBoAEvZc,WSB-TV Channel 2 Action News,4.5,224,"[point_of_interest, establishment]",30309,"1601 West Peachtree Street Northeast, Atlanta",point_of_interest
4,33.807117,-84.384343,GA,ChIJS6x_52YE9YgRvjzX8rkA3xI,"Influence Health - Atlanta, GA",4.5,6,"[point_of_interest, establishment]",30309,"120 Ottley Drive Northeast suite 150, Atlanta",point_of_interest
...,...,...,...,...,...,...,...,...,...,...,...
5278,30.800110,-84.920406,GA,ChIJ4ba6nYUq7YgRSUgZ-9bRWG4,Papa Doc's Lakeside Grill,4.1,14,"[restaurant, point_of_interest, food, establis...",32460,"3371 Georgia 253, Donalsonville",restaurant
5279,30.800136,-84.920361,GA,ChIJIWan0f9H7YgRJRTfaocDBWQ,Pop's Dockside Grill,4.6,180,"[restaurant, point_of_interest, food, establis...",32460,"3371 Georgia 253, Donalsonville",restaurant
5280,33.451486,-81.925004,GA,ChIJCc0PltvL-YgRwMyapR2fSIQ,Subway,4.0,325,"[meal_takeaway, restaurant, food, point_of_int...",29842,"724 Laney Walker Boulevard Ext, Augusta",meal_takeaway
5281,34.978807,-85.348016,GA,ChIJ9XcT4ptcYIgRK4bk_0yBwVc,The Lookout Mountain Club,4.6,115,"[restaurant, food, point_of_interest, health, ...",37350,"1201 Fleetwood Drive, Lookout Mountain",restaurant


<IPython.core.display.Javascript object>

In [41]:
poi_df['primary_type'].value_counts()



restaurant                 3058
meal_takeaway              1006
point_of_interest           645
bar                         499
lodging                     335
cafe                        269
church                      263
meal_delivery               224
convenience_store           186
gas_station                 179
store                       128
general_contractor           98
secondary_school             77
car_repair                   71
doctor                       62
drugstore                    55
post_office                  50
grocery_or_supermarket       49
local_government_office      48
health                       43
home_goods_store             42
bakery                       42
bank                         41
department_store             40
tourist_attraction           40
finance                      39
school                       37
park                         34
clothing_store               33
hospital                     29
real_estate_agency           27
insuranc

<IPython.core.display.Javascript object>

In [43]:

# Create an S3 resource instead of using the S3 client that was used earlier
s3R = boto3.resource('s3')

# Convert the poi_df DataFrame to a csv buffer in memory
csv_buffer = io.StringIO()
poi_df.to_csv(csv_buffer, index=False)

# Naming the file for AWS storage
file_name = 'poi_combined_haystack.csv'

# Uploading the file to S3 and printing the response
response = s3R.Object(bucket_name, file_name).put(Body=csv_buffer.getvalue())
print(f"File uploaded to S3 with ETag: {response['ETag']}") # Checking the status to confirm it was uploaded


File uploaded to S3 with ETag: "00852aa0dcf8a578eab3ca4b4eb892c1"


<IPython.core.display.Javascript object>